# **Topic Modeling**
##(2) TF-IDF & NMF
#### Using with TF-IDf & NMF models from Scikitlearn

# 0. Road datasets


In [ ]:
# Import the necessary libraries
import pandas as pd
from nltk.corpus import stopwords
from nltk.util import ngrams
from collections import Counter
# pip install wordcloud matplotlib
from wordcloud import WordCloud
import matplotlib.pyplot as plt


In [ ]:
import pandas as pd

data = pd.read_csv('/Users/leahshin/Documents/myProject/hotel_review_with_region.csv')

In [ ]:
data.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,name,region,sub_region
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/17,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Russia,Europe,Eastern Europe
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/17,7.7,Hotel Arena,Ireland,No Negative,0,1403,...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Ireland,Europe,Northern Europe
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/17,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,Australia,Oceania,Australia and New Zealand
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/17,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,United Kingdom,Europe,Northern Europe
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/17,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,...,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,New Zealand,Oceania,Australia and New Zealand


In [ ]:
set(data.region)

{'Africa', 'Americas', 'Antarctica', 'Asia', 'Europe', 'Oceania'}

In [ ]:
set(data.sub_region)

{'Antarctica',
 'Australia and New Zealand',
 'Central Asia',
 'Eastern Africa',
 'Eastern Asia',
 'Eastern Europe',
 'Latin America and the Caribbean',
 'Melanesia',
 'Micronesia',
 'Middle Africa',
 'Northern Africa',
 'Northern America',
 'Northern Europe',
 'Polynesia',
 'South-eastern Asia',
 'Southern Africa',
 'Southern Asia',
 'Southern Europe',
 'Western Africa',
 'Western Asia',
 'Western Europe'}

In [ ]:
# check all columns
data.columns.values

array(['Unnamed: 0', 'Hotel_Address', 'Additional_Number_of_Scoring',
       'Review_Date', 'Average_Score', 'Hotel_Name',
       'Reviewer_Nationality', 'Negative_Review',
       'Review_Total_Negative_Word_Counts', 'Total_Number_of_Reviews',
       'Positive_Review', 'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
       'Tags', 'days_since_review', 'lat', 'lng', 'name', 'region',
       'sub_region'], dtype=object)

In [ ]:
# Merge negative and positive reviews into one column
data['Merged_Review'] = data['Negative_Review'] + " " + data['Positive_Review']

# 1. Preprocess Datasets

In [ ]:
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Define a function to preprocess text (including sentimental words removal)

stop_words = set(stopwords.words('english'))
stop_words_nouns = ["hotel", "place", "room", "5", "4", "3", "2", "1", "wa", "u", "one", "two", "everything", "nothing", "thing"]
stop_words_verbs = ["enjoy", "would", "like", "could", "get", "stay", "visit", "go", "think", "make"]
stop_words_adverbs = ["highly", "within", "really", "well", "even", "though", "bit", "little", "extremely",
                 "definitely", "much", "also", "always", "never", "often", "usually"]
stop_words_adjectives =["great", "positive", "negative", "good", "nice", "lovely", "fantastic", "excellent",
                 "amazing", "wonderful", "horrible", "terrible", "disappointing", "awful", "poor", "super"]

# Not Included: "spacious", "friendly", "helpful", "close", "staff", "experience", which could be meaningful for topics.

# Merge all custom stopwords lists
all_stopwords = list(stop_words.union(set(stop_words_nouns)))
all_stopwords += stop_words_verbs + stop_words_adverbs + stop_words_adjectives

# Convert the custom stopwords list to a set for faster membership checking
all_stopwords = set(all_stopwords)


In [ ]:
def preprocess_text(text):
    if isinstance(text, str):
        # Remove punctuation and convert to lowercase
        text = text.translate(str.maketrans('', '', string.punctuation)).lower()
        # Tokenize the text
        tokens = word_tokenize(text)

        # Lemmatize the tokens
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]

        # Remove stopwords
        filtered_tokens = [word for word in lemmatized_tokens if word not in all_stopwords]
        return filtered_tokens
    else:
        # Handle non-string values (e.g., float) gracefully by returning an empty list
        return []

# Apply text preprocessing to the merged reviews
data['Merged_Review'] = data['Merged_Review'].apply(preprocess_text)

In [ ]:
data['Merged_Review'] = data['Merged_Review'].astype(str)

<br><br>

#2. TF-IDF & NMF Modeling

In [ ]:
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# TF-IDF with custom stopwords
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), max_features=1000, stop_words=all_stopwords_list)
tfidf_matrix = tfidf_vectorizer.fit_transform(data['Merged_Review'])

# Get the feature names (unigrams)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a DataFrame to store TF-IDF scores for each sub-region and unigram
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
tfidf_df['sub_region'] = data['sub_region']

# Calculate the top TF-IDF weighted unigrams for each sub-region
top_unigrams_by_subregion = {}
sub_regions = data['sub_region'].unique()


In [ ]:
from sklearn.decomposition import NMF

# Choose the number of topics
num_topics = 10

# Initialize and fit the NMF model
nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_matrix = nmf_model.fit_transform(tfidf_matrix)


# Extract the top 7 topics for each sub-region:


In [ ]:
# Add the NMF topic matrix to your DataFrame
data['NMF_Topics'] = nmf_matrix.tolist()

# Group data by sub-region
sub_region_groups = data.groupby('sub_region')

# Initialize a dictionary to store top topics for each sub-region
top_topics_by_sub_region = {}

# Iterate through sub-regions and extract top topics
for sub_region, group in sub_region_groups:
    sub_region_topics = group['NMF_Topics'].sum()
    top_topics = [i for i in range(num_topics) if sub_region_topics[i] == max(sub_region_topics)]
    top_topics_by_sub_region[sub_region] = top_topics[:7]

# Print the top 7 topics for each sub-region
for sub_region, top_topics in top_topics_by_sub_region.items():
    print(f"Top 7 topics for {sub_region}: {top_topics}")


Sub-Region: Eastern Europe
Unigram: breakfast
Unigram: location
Unigram: staff
Unigram: clean
Unigram: bed
Unigram: perfect
Unigram: small
Unigram: friendly
Unigram: comfortable
Unigram: close
Unigram: parking
Unigram: bathroom
Unigram: station
Unigram: helpful
Unigram: center
Unigram: city
Unigram: metro
Unigram: view
Unigram: big
Unigram: price

Sub-Region: Northern Europe
Unigram: staff
Unigram: location
Unigram: breakfast
Unigram: bed
Unigram: friendly
Unigram: helpful
Unigram: clean
Unigram: comfortable
Unigram: small
Unigram: bar
Unigram: service
Unigram: comfy
Unigram: bathroom
Unigram: view
Unigram: facility
Unigram: shower
Unigram: food
Unigram: night
Unigram: perfect
Unigram: reception

Sub-Region: Australia and New Zealand
Unigram: location
Unigram: staff
Unigram: helpful
Unigram: breakfast
Unigram: friendly
Unigram: bed
Unigram: comfortable
Unigram: small
Unigram: close
Unigram: clean
Unigram: station
Unigram: restaurant
Unigram: service
Unigram: bathroom
Unigram: facility


In [ ]:
# Get the feature names (words) from the TF-IDF vectorizer
feature_names = tfidf_vectorizer.get_feature_names_out()

# Get the topic-word matrix from the NMF model
topic_word_matrix = nmf_model.components_

# Defining top words that we would like to check
num_top_words = 20

# Iterate through topics and print the top words
for topic_idx, topic in enumerate(topic_word_matrix):
    top_words_idx = topic.argsort()[-num_top_words:][::-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(f"Topic {topic_idx + 1}: {' '.join(top_words)}")


Topic 1: staff helpful reception polite pleasant welcoming attentive kind food facility professional beautiful especially brilliant rude courteous desk recommend help friendliness
Topic 2: location staff facility price central size value cleanliness convenient comfort wifi money old quiet expensive ideal near superb noisy need
Topic 3: bed comfortable comfy shower bathroom pillow double size hard uncomfortable large big quiet extra single soft spacious bath huge twin
Topic 4: breakfast expensive choice included buffet price selection delicious variety better quality cooked egg option cold continental coffee fresh hot english
Topic 5: close station bar restaurant view night area time bathroom walk shower day metro city free check facility reception floor price
Topic 6: small bathroom size quite shower space double side bedroom old single elevator lift people toilet standard tiny price window rather
Topic 7: clean comfortable modern spacious quiet tidy new bathroom big value wifi facilit

<br>

### Based on the topics that we extracted from the NMF model, we could draw a conclusion that TF-IDF & NMF does not perform the best topic modeling process, which means the trained topics were not useful for our project. So we will move on to the next methodology, LDA analysis with Gensim.